## Patches for classification

In [1]:
import sys
import os

# Chemin du dossier parent
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

# Exemple d'import
import config
import utils

import data.dataset as dataset
import data.transform as T_mtsk


/home/tibia/Projet_Hemorragie/hemorragie-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
train_data=dataset.get_classification_data(split="train")
train_transforms, val_transforms = T_mtsk.TaskBasedTransform_V2(keys=["image", "label"]), T_mtsk.TaskBasedValTransform_V2(keys=["image", "label"])


>>> TaskBasedTransform initialized
>>> TaskBasedTransform initialized


/home/tibia/Projet_Hemorragie/hemorragie-env/lib/python3.12/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


In [10]:
print (f"longueur du jeu de données: {len(train_data)}")
sample = train_data[0]   # on récupère le premier échantillon
print(f" : clé de l'image : {sample.keys()}") 
print(f" : type de l'image : {type(sample['image'])}")
print(f"label : {sample['label']}")

transformed_sample = train_transforms(sample)

# Vérifier le résultat
print(type(transformed_sample))
print(transformed_sample["image"].shape)
print(transformed_sample["label"])

longueur du jeu de données: 1274
 : clé de l'image : dict_keys(['image', 'label', 'task'])
 : type de l'image : <class 'str'>
label : [1. 0. 1. 1. 1. 0.]
<class 'dict'>
torch.Size([1, 96, 96, 96])
metatensor([1., 0., 1., 1., 1., 0.])


In [35]:
import torch
import numpy as np
from monai.transforms import MapTransform
from monai.utils.misc import ensure_tuple_rep
from typing import Dict, Hashable, Sequence, Union


class SlidingWindowCropd(MapTransform):
    """
    Transformation MONAI pour créer des patches en sliding window.
    Compatible avec les pipelines MONAI existantes.
    """
    
    def __init__(
        self,
        keys: Union[Hashable, Sequence[Hashable]],
        roi_size: Union[int, Sequence[int]],
        overlap: float = 0.0,
        allow_missing_keys: bool = False,
    ):
        """
        Args:
            keys: clés à transformer (ex: ["image"])
            roi_size: taille des patches (ex: (96, 96, 96))
            overlap: pourcentage de chevauchement entre patches (0.0 = pas de chevauchement)
            allow_missing_keys: permettre les clés manquantes
        """
        super().__init__(keys, allow_missing_keys)
        self.roi_size = ensure_tuple_rep(roi_size, 3)  # Assure 3D
        self.overlap = overlap
        
    def _compute_patch_positions(self, image_shape: Sequence[int]) -> list:
        """Calcule les positions de départ de chaque patch"""
        positions = []
        
        # Pour chaque dimension
        starts = []
        for i, (img_size, patch_size) in enumerate(zip(image_shape[-3:], self.roi_size)):
            if img_size <= patch_size:
                # Si l'image est plus petite que le patch, un seul patch centré
                starts.append([max(0, (img_size - patch_size) // 2)])
            else:
                # Calcule le stride avec overlap
                stride = int(patch_size * (1 - self.overlap))
                stride = max(1, stride)  # Au moins 1
                
                start_positions = []
                start = 0
                while start + patch_size <= img_size:
                    start_positions.append(start)
                    start += stride
                
                print(f"start:{start}")
                print(f"start_pos{start_positions}")
                # Assure qu'on couvre bien la fin de l'image
                if start_positions[-1] + patch_size < img_size:
                  
                    
                    start_positions.append(img_size - patch_size)
                # starts liste de liste ou chaque liste contient points de départ patch selon dimension
                starts.append(start_positions) 
                
        print(f"starts at the end : {starts}")
        # Génère toutes les combinaisons de positions
        for z_start in starts[0]:
            for y_start in starts[1]:
                for x_start in starts[2]:
                    positions.append((z_start, y_start, x_start))
        
        return positions
    
    def _extract_patches(self, image: torch.Tensor, positions: list) -> torch.Tensor:
        """Extrait tous les patches de l'image"""
        patches = []
        
        for z_start, y_start, x_start in positions:
            z_end = z_start + self.roi_size[0]
            y_end = y_start + self.roi_size[1]
            x_end = x_start + self.roi_size[2]
            
            patch = image[..., z_start:z_end, y_start:y_end, x_start:x_end]
            patches.append(patch)
        
        # Stack tous les patches : [N_patches, C, H, W, D]
        return torch.stack(patches, dim=0)
    
    def __call__(self, data: Dict) -> Dict:
        d = dict(data)
        
        for key in self.key_iterator(d):
            image = d[key]
            
            # Assure que c'est un tensor
            if not isinstance(image, torch.Tensor):
                image = torch.tensor(image)
            
            # Calcule les positions des patches
            positions = self._compute_patch_positions(image.shape)
            
            # Extrait les patches
            patches = self._extract_patches(image, positions)
            
            # Stocke les patches et les métadonnées
            d[key] = patches
            d[f"{key}_patch_positions"] = positions
            d[f"{key}_original_shape"] = image.shape
            d[f"{key}_n_patches"] = len(positions)
        
        return d


# Exemple d'utilisation dans ta pipeline de classification
# if __name__ == "__main__":
#     # Test rapide
#     import torch
    
#     # Simule une image 3D
#     fake_data = {
#         "image": torch.randn(1, 128, 128, 96),  # [C, H, W, D]
#         "task": "classification"
#     }
    
#     # Applique la transformation
#     sliding_window = SlidingWindowCropd(
#         keys=["image"], 
#         roi_size=(96, 96, 96), 
#         overlap=0.2
#     )
    
#     result = sliding_window(fake_data)
    
#     print(f"Original shape: {fake_data['image'].shape}")
#     print(f"Patches shape: {result['image'].shape}")
#     print(f"Number of patches: {result['image_n_patches']}")
#     print(f"Patch positions: {result['image_patch_positions'][:3]}...")  # Premiers patches

In [19]:
from monai.utils.misc import ensure_tuple_rep
# test ensure_tuple_rep
ensure_tuple_rep((1,1,1), 3)


#test compute_patch_position
fake_data = {
        "image": torch.randn(1, 128, 128, 96),  # [C, H, W, D]
        "task": "classification",
        "label": [1,0,0,1,0,1]
    }


In [37]:
image_shape = (1, 512, 512, 128)
roi_size = (96, 96, 96)

# We are only interested in the last 3 dimensions
spatial_dims_shape = image_shape[-3:] # -> (128, 128, 96)
print(f"Spatial dimensions of image: {spatial_dims_shape}")
print(f"Patch dimensions (roi_size): {roi_size}\n")

# Let's see what zip() does
zipped_dimensions = zip(spatial_dims_shape, roi_size)
print(f"Zipped dimensions (iterator): {list(zipped_dimensions)}\n")
# After converting to a list for printing, the iterator is exhausted.

# Let's restart the zip to show what happens in the loop
zipped_dimensions_for_loop = zip(spatial_dims_shape,roi_size)
for i, (img_size, patch_size) in enumerate(zipped_dimensions_for_loop):
    print(f"--- Iteration {i} ---")
    print(f"  Current dimension index (i): {i}")
    print(f"  Image size for this dimension (img_size): {img_size}")
    print(f"  Patch size for this dimension (patch_size): {patch_size}")
    print(f"  This corresponds to: ")
    if i == 0:
        print("    The first spatial dimension (e.g., Depth)")
    elif i == 1:
        print("    The second spatial dimension (e.g., Height)")
    elif i == 2:
        print("    The third spatial dimension (e.g., Width)")
    print()

Spatial dimensions of image: (512, 512, 128)
Patch dimensions (roi_size): (96, 96, 96)

Zipped dimensions (iterator): [(512, 96), (512, 96), (128, 96)]

--- Iteration 0 ---
  Current dimension index (i): 0
  Image size for this dimension (img_size): 512
  Patch size for this dimension (patch_size): 96
  This corresponds to: 
    The first spatial dimension (e.g., Depth)

--- Iteration 1 ---
  Current dimension index (i): 1
  Image size for this dimension (img_size): 512
  Patch size for this dimension (patch_size): 96
  This corresponds to: 
    The second spatial dimension (e.g., Height)

--- Iteration 2 ---
  Current dimension index (i): 2
  Image size for this dimension (img_size): 128
  Patch size for this dimension (patch_size): 96
  This corresponds to: 
    The third spatial dimension (e.g., Width)



In [38]:
transform_no_overlap = SlidingWindowCropd(keys=["image"], roi_size=(96, 96, 96), overlap=0.0)
transform_with_overlap = SlidingWindowCropd(keys=["image"], roi_size=(96, 96, 96), overlap=0.25)

# Définir la forme de l'image que vous voulez tester
image_shape = (1, 512, 512, 128) # [C, H, W, D]

# Tester la méthode _compute_patch_positions sans chevauchement
print("--- Test sans chevauchement (overlap=0.0) ---")
positions_no_overlap = transform_no_overlap._compute_patch_positions(image_shape)
print(f"Forme de l'image: {image_shape[1:]}")
print(f"Taille du patch: {transform_no_overlap.roi_size}")
print(f"Nombre total de patches: {len(positions_no_overlap)}")
print(f"Positions de départ des patches:\n{positions_no_overlap}\n")

# Tester la méthode _compute_patch_positions avec chevauchement
print("--- Test avec chevauchement (overlap=0.25) ---")
positions_with_overlap = transform_with_overlap._compute_patch_positions(image_shape)
print(f"Forme de l'image: {image_shape[1:]}")
print(f"Taille du patch: {transform_with_overlap.roi_size}")
print(f"Nombre total de patches: {len(positions_with_overlap)}")
print(f"Positions de départ des patches:\n{positions_with_overlap}")

--- Test sans chevauchement (overlap=0.0) ---
start:480
start_pos[0, 96, 192, 288, 384]
start:480
start_pos[0, 96, 192, 288, 384]
start:96
start_pos[0]
starts at the end : [[0, 96, 192, 288, 384, 416], [0, 96, 192, 288, 384, 416], [0, 32]]
Forme de l'image: (512, 512, 128)
Taille du patch: (96, 96, 96)
Nombre total de patches: 72
Positions de départ des patches:
[(0, 0, 0), (0, 0, 32), (0, 96, 0), (0, 96, 32), (0, 192, 0), (0, 192, 32), (0, 288, 0), (0, 288, 32), (0, 384, 0), (0, 384, 32), (0, 416, 0), (0, 416, 32), (96, 0, 0), (96, 0, 32), (96, 96, 0), (96, 96, 32), (96, 192, 0), (96, 192, 32), (96, 288, 0), (96, 288, 32), (96, 384, 0), (96, 384, 32), (96, 416, 0), (96, 416, 32), (192, 0, 0), (192, 0, 32), (192, 96, 0), (192, 96, 32), (192, 192, 0), (192, 192, 32), (192, 288, 0), (192, 288, 32), (192, 384, 0), (192, 384, 32), (192, 416, 0), (192, 416, 32), (288, 0, 0), (288, 0, 32), (288, 96, 0), (288, 96, 32), (288, 192, 0), (288, 192, 32), (288, 288, 0), (288, 288, 32), (288, 384, 0

In [42]:
import data.dataset as dataset

import nibabel as nib
import data.dataset as dataset
import os

# Assuming your get_classification_data function is correctly defined and imports necessary modules.

# Get the list of data dictionaries
train_data = dataset.get_classification_data(split="train")

# Check if the list is not empty
if train_data:
    # Get the dictionary for the first image
    first_image_dict = train_data[9]
    
    # Get the image file path from the dictionary
    image_path = first_image_dict["image"]
    
    # Check if the file exists before trying to load it
    if os.path.exists(image_path):
        try:
            # Load the NIfTI file using nibabel
            img_obj = nib.load(image_path)
            
            # Get the data dimensions from the NIfTI object
            image_dimensions = img_obj.shape
            
            print(f"File path: {image_path}")
            print(f"Dimensions of the first image: {image_dimensions}")
            
        except Exception as e:
            print(f"Error loading image file {image_path}: {e}")
    else:
        print(f"File not found: {image_path}")
else:
    print("No data found in the train dataset.")

File path: /home/tibia/Projet_Hemorragie/MBH_label_case/ID_45be38b3_ID_fa1ed92647.nii.gz
Dimensions of the first image: (512, 512, 27)


In [7]:

import data.dataset as dataset
import config
import nibabel as nib
import data.dataset as dataset
import os

seg_data=dataset.get_segmentation_data(split = 'train')
cls_data=dataset.get_classification_data(split = 'train')

for i in range(5):
    image_dict= seg_data[i]
    image_path= image_dict["image"]
    img_obj = nib.load(image_path)
    image_dimensions = img_obj.shape
    print(f"File path: {image_path}")

    print(f"Dimensions of the first segmentation image: {image_dimensions}")
    
import nibabel as nib
import os
import collections
import data.dataset as dataset
from typing import Dict, List

def count_images_by_slices(dataset: List[Dict], dataset_name: str) -> None:
    """
    Counts and prints the number of images per slice count for a given dataset.
    """
    slice_counts = collections.defaultdict(int)
    
    # Iterate through each dictionary in the dataset list
    for data_dict in dataset:
        image_path = data_dict.get("image")
        if not image_path or not os.path.exists(image_path):
            print(f"Warning: Image file not found at {image_path}")
            continue
            
        try:
            # Load the image and get its dimensions
            img_obj = nib.load(image_path)
            # The number of slices is the last dimension in the (D, H, W) tuple
            num_slices = img_obj.shape[-1]
            slice_counts[num_slices] += 1
            
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")

    # Print the results in a readable format
    print(f"\n--- {dataset_name} Dataset Summary ---")
    if not slice_counts:
        print("No images were successfully processed.")
        return
        
    for num_slices, count in sorted(slice_counts.items()):
        print(f"Number of images with {num_slices} slices: {count}")
    print("-" * 35)

# --- Main execution ---
seg_data = dataset.get_segmentation_data(split='train')
cls_data = dataset.get_classification_data(split='train')

count_images_by_slices(seg_data, "Segmentation Train")
count_images_by_slices(cls_data, "Classification Train")

File path: /home/tibia/Projet_Hemorragie/Seg_hemorragie/split_MONAI/train/img/ID_0237f3c9_ID_40015688b9.nii.gz
Dimensions of the first segmentation image: (512, 512, 27)
File path: /home/tibia/Projet_Hemorragie/Seg_hemorragie/split_MONAI/train/img/ID_02b882cc_ID_a4892e60ae.nii.gz
Dimensions of the first segmentation image: (512, 512, 30)
File path: /home/tibia/Projet_Hemorragie/Seg_hemorragie/split_MONAI/train/img/ID_02f779fb_ID_c4d7f33559.nii.gz
Dimensions of the first segmentation image: (512, 512, 31)
File path: /home/tibia/Projet_Hemorragie/Seg_hemorragie/split_MONAI/train/img/ID_041e6601_ID_1fa1d20bba.nii.gz
Dimensions of the first segmentation image: (512, 512, 32)
File path: /home/tibia/Projet_Hemorragie/Seg_hemorragie/split_MONAI/train/img/ID_0492041f_ID_b4fcf2f799.nii.gz
Dimensions of the first segmentation image: (512, 512, 36)

--- Segmentation Train Dataset Summary ---
Number of images with 22 slices: 2
Number of images with 23 slices: 1
Number of images with 25 slices: 3
N

In [5]:
import torch
import nibabel as nib
import monai.transforms as T
from monai.data import decollate_batch
import os
import data.dataset as dataset
import config

# Use the first image from your dataset
seg_data = dataset.get_segmentation_data(split='train')
image_dict = seg_data[0]

# Define a simple transform pipeline to load the image
simple_transform = T.Compose([
    T.LoadImaged(keys=["image", "label"]),
    T.EnsureChannelFirstd(keys=["image", "label"]),
])

# Apply the transform to the single image dictionary
transformed_data = simple_transform(image_dict)

# The loaded data is a MetaTensor. You can access its metadata.
image_meta = transformed_data['image'].meta

# Print the metadata
print("--- Image Metadata ---")
print(f"File Name: {image_meta.get('filename_or_obj')}")
print(f"Affine Matrix:\n{image_meta.get('affine')}")
print(f"affine shape: {image_meta.get('affine').shape}")
print(f"Original Spatial Shape: {image_meta.get('spatial_shape')}")
print(f"Header:\n{image_meta.get('original_affine')}")
print("-" * 25)

--- Image Metadata ---
File Name: /home/tibia/Projet_Hemorragie/Seg_hemorragie/split_MONAI/train/img/ID_0237f3c9_ID_40015688b9.nii.gz
Affine Matrix:
tensor([[-4.8828e-01,  0.0000e+00,  0.0000e+00,  1.2500e+02],
        [ 0.0000e+00, -4.6168e-01, -1.7216e+00,  1.6214e+02],
        [ 0.0000e+00, -1.5897e-01,  4.9999e+00,  2.5580e+01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]],
       dtype=torch.float64)
affine shape: torch.Size([4, 4])
Original Spatial Shape: [512 512  27]
Header:
[[-4.88281012e-01  0.00000000e+00  0.00000000e+00  1.25000000e+02]
 [ 0.00000000e+00 -4.61678816e-01 -1.72160287e+00  1.62141907e+02]
 [ 0.00000000e+00 -1.58968604e-01  4.99990287e+00  2.55799370e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
-------------------------
